In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Data Visualization

In [ ]:
data_soybean = pd.read_csv('/kaggle/input/crop-price-prediction/dataset_42_soybean.csv')
data_yield = pd.read_csv('/kaggle/input/crop-price-prediction/corn yield.csv')
# data_yield

In [ ]:
data_soybean

In [ ]:
import re
from tqdm import tqdm
tqdm.pandas()

def empty_string(x):
    return ''.join(list(filter(None, re.split(' ',x))))

for i in data_soybean.columns:
    data_soybean[i] = data_soybean[i].progress_apply(empty_string)

In [ ]:
data_soybean.loc[0,'precip']

In [ ]:
label = []
length = []
columns = []

for i in data_soybean.columns:
    label.append(i)
    length.append(data_soybean[i].unique())
    columns.append(data_soybean[i].nunique())
    
data_analysis = pd.DataFrame({'Label': label, 'Unique No': length, 'Unique List': columns, 
                              'Example': data_soybean.loc[1].tolist()}) 
data_analysis #.sort_values(by=['Unique List'])

In [ ]:
column_name = 'date' 

import plotly.graph_objects as go
go.Figure(data=[go.Pie(labels=data_soybean[column_name].value_counts().index, 
                       values=data_soybean[column_name].value_counts().values)])


In [ ]:
column_name_1 = 'date' 
column_name_2 = 'temp' 
column_name_3 = 'precip' 
column_name_4 = 'plant-growth' 

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.offline import plot


fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=(column_name_1, column_name_2, 
                                    column_name_3, column_name_4),
                    specs=[[{"type": "pie"}, {"type": "pie"}],
                           [{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(go.Pie(
     values=data_soybean[column_name_1].value_counts().values,
     labels=data_soybean[column_name_1].value_counts().index,
     domain=dict(x=[0, 0.5]),
     name= column_name_1), 
     row=1, col=1)

fig.add_trace(go.Pie(
     values=data_soybean[column_name_2].value_counts().values,
     labels=data_soybean[column_name_2].value_counts().index,
     domain=dict(x=[0.5, 1.0]),
     name= column_name_2),
     row=1, col=2)

fig.add_trace(go.Pie(
     values=data_soybean[column_name_3].value_counts().values,
     labels=data_soybean[column_name_3].value_counts().index,
     domain=dict(x=[0, 0.5]),
     name= column_name_1), 
     row=2, col=1)

fig.add_trace(go.Pie(
     values=data_soybean[column_name_4].value_counts().values,
     labels=data_soybean[column_name_4].value_counts().index,
     domain=dict(x=[0.5, 1.0]),
     name= column_name_2),
     row=2, col=2)



#fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))
fig.show()

In [ ]:
data_soybean.loc[:20]

# Machine Learning (Prediction Approach with Bayesian Network)

In [ ]:
print(__doc__)
import warnings

# Graph Visualization
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats.stats import pearsonr
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import SVC

# ML Metrics
from sklearn.metrics import accuracy_score, recall_score, precision_score

%matplotlib inline
# Suppress warnings
warnings.filterwarnings("ignore")

In [ ]:
def cross_validate(estimator, train, validation):
    X_train = train[0]
    Y_train = train[1]
    X_val = validation[0]
    Y_val = validation[1]
    train_predictions = classifier.predict(X_train)
    train_accuracy = accuracy_score(train_predictions, Y_train)
    train_recall = recall_score(train_predictions, Y_train, average='micro')
    train_precision = precision_score(train_predictions, Y_train, average='micro')

    val_predictions = classifier.predict(X_val)
    val_accuracy = accuracy_score(val_predictions, Y_val)
    val_recall = recall_score(val_predictions, Y_val, average='micro')
    val_precision = precision_score(val_predictions, Y_val, average='micro')

    print('Model metrics')
    print('Accuracy  Train: %.2f, Validation: %.2f' % (train_accuracy, val_accuracy))
    print('Recall    Train: %.2f, Validation: %.2f' % (train_recall, val_recall))
    print('Precision Train: %.2f, Validation: %.2f' % (train_precision, val_precision))

In [ ]:
import pandas as pd
cat_list = ['Sun', 'Sun', 'Wed', 'Mon', 'Mon']
encoded_data, mapping_index = pd.Series(cat_list).factorize()
encoded_data, mapping_index

In [ ]:
cat_list = ['?','lt-norm','norm','gt-norm']
encoded_data, mapping_index = pd.Series(cat_list).factorize()
encoded_data, mapping_index

In [ ]:
#Mapping the labels 
month = ['?','april','may','june','july','august','september','october']
temp = ['?','lt-norm','norm','gt-norm']
precip = ['?','lt-norm','norm','gt-norm']
growth = ['?','abnorm','norm']


month_map = {month[i]: i for i in range(len(month))}
temp_map = {temp[i]: i for i in range(len(temp))}
precip_map = {precip[i]: i for i in range(len(precip))}
growth_map = {growth[i]: i for i in range(len(growth))}

month_map

In [ ]:
growth_map

In [ ]:
encoded_data_month, mapping_index_month = pd.Series(month).factorize()
encoded_data_temp, mapping_index_temp = pd.Series(temp).factorize()
encoded_data_precip, mapping_index_precip = pd.Series(precip).factorize()
encoded_data_growth, mapping_index_growth = pd.Series(growth).factorize()

dataset = data_soybean[['date','precip','temp','plant-growth']]
dataset['date'] = dataset['date'].map(month_map)
dataset['temp'] = dataset['temp'].map(temp_map)
dataset['precip'] = dataset['precip'].map(precip_map)
dataset['plant-growth'] = dataset['plant-growth'].map(growth_map)
dataset

In [ ]:
labels = dataset['plant-growth'].values
dataset_final = dataset.drop(['plant-growth'], axis=1, inplace=True)

classifier = GaussianNB()
X_train, X_test, Y_train, Y_test = train_test_split(dataset, labels, test_size=0.2, random_state=1)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=1)
X_train1, X_train2, Y_train1, Y_train2 = train_test_split(X_train, Y_train, test_size=0.3, random_state=12)

classifier.fit(X_train2, Y_train2)

In [ ]:
print('Metrics with only 30% of train data')
cross_validate(classifier, (X_train, Y_train), (X_val, Y_val))

In [ ]:
classifier.partial_fit(X_train1, Y_train1)

In [ ]:
print('Metrics with the remaining 70% of train data')
cross_validate(classifier, (X_train, Y_train), (X_val, Y_val))

In [ ]:
print('Probability of each class')
print('Survive = 0: %.2f' % classifier.class_prior_[0])
print('Survive = 1: %.2f' % classifier.class_prior_[1])

In [ ]:
print('Mean of each feature per class')
print('Factors')
print('Survive = 0: %s' % classifier.theta_[0])
print('Survive = 1: %s' % classifier.theta_[1])

In [ ]:
print('Variance of each feature per class')
print('Survive = 0: %s' % classifier.sigma_[0])
print('Survive = 1: %s' % classifier.sigma_[1])

In [ ]:
test_predictions = classifier.predict(X_test)
test_predictions

In [ ]:
Y_test

In [ ]:
new_df = pd.DataFrame({'Real': Y_test, 'Prediction': test_predictions})
new_df['True'] = new_df['Real'] == new_df['Prediction']
new_df

In [ ]:
accuracy = round(sum(new_df['True'])/len(new_df)*100,2)

print('Accuracy: %s percent' % accuracy)

In [ ]:
classifier.get_params()

In [ ]:
def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : estimator instance
        An estimator instance implementing `fit` and `predict` methods which
        will be cloned for each validation.

    title : str
        Title for the chart.

    X : array-like of shape (n_samples, n_features)
        Training vector, where ``n_samples`` is the number of samples and
        ``n_features`` is the number of features.

    y : array-like of shape (n_samples) or (n_samples, n_features)
        Target relative to ``X`` for classification or regression;
        None for unsupervised learning.

    axes : array-like of shape (3,), default=None
        Axes to use for plotting the curves.

    ylim : tuple of shape (2,), default=None
        Defines minimum and maximum y-values plotted, e.g. (ymin, ymax).

    cv : int, cross-validation generator or an iterable, default=None
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, default=None
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like of shape (n_ticks,)
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the ``dtype`` is float, it is regarded
        as a fraction of the maximum size of the training set (that is
        determined by the selected validation method), i.e. it has to be within
        (0, 1]. Otherwise it is interpreted as absolute sizes of the training
        sets. Note that for classification the number of samples usually have
        to be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 15))

X, y = dataset, labels

title = "Learning Curves (Naive Bayes)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = GaussianNB()
plot_learning_curve(estimator, title, X, y, axes=axes[:, 0], ylim=(0.5, 0.9),
                    cv=cv, n_jobs=4)

title = r"Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator_2 = SVC(gamma=0.001)
plot_learning_curve(estimator_2, title, X, y, axes=axes[:, 1], ylim=(0.7, 1.01),
                    cv=cv, n_jobs=4)

plt.show()

In [ ]:
data_soybean['Value'] = 1
data_soybean.to_csv('dataset.csv')

# save our model code
classifier.export('classifier.py')

# print the model code to see what it says
!cat classifier.py

https://www.kaggle.com/kerneler/starter-crop-price-prediction-e16a7dd4-a

In R: https://www.kaggle.com/plantdevo/crop-price-models

https://www.kaggle.com/robinwilson/crop-yield-prediction